In [1]:
import tkinter as tk
from tkinter import messagebox, simpledialog
import json
from datetime import datetime

# Data persistence files will be defined for saving and loading data
PRODUCTS_FILE = "products.json"
ORDERS_FILE = "orders.json"
INVOICES_FILE = "invoices.csv"

# Data will be loaded when the program starts
def load_data(filename):
    try:
        with open(filename, 'r') as file:
            return json.load(file)
    except FileNotFoundError:
        return []

# Data will be saved into the specified file
def save_data(filename, data):
    with open(filename, 'w') as file:
        json.dump(data, file, indent=4)
        

# Initializing products and orders
products = load_data(PRODUCTS_FILE)
orders = load_data(ORDERS_FILE)

# Admin Functionalities
# Admin will add a product
def add_product():
    product_id = simpledialog.askstring("Add Product", "Enter Product ID:")
    name = simpledialog.askstring("Add Product", "Enter Product Name:")
    category = simpledialog.askstring("Add Product", "Enter Product Category:")
    price = float(simpledialog.askstring("Add Product", "Enter Product Price:"))
    stock = int(simpledialog.askstring("Add Product", "Enter Stock Quantity:"))

    products.append({"id": product_id, "name": name, "category": category, "price": price, "stock": stock})
    save_data(PRODUCTS_FILE, products)
    messagebox.showinfo("Success", "Product added successfully!")

# Admin will update a product
def update_product():
    product_id = simpledialog.askstring("Update Product", "Enter Product ID to update:")
    for product in products:
        if product["id"] == product_id:
            product["name"] = simpledialog.askstring("Update Product", "Enter New Name:") or product["name"]
            product["category"] = simpledialog.askstring("Update Product", "Enter New Category:") or product["category"]
            product["price"] = float(simpledialog.askstring("Update Product", "Enter New Price:") or product["price"])
            product["stock"] = int(simpledialog.askstring("Update Product", "Enter New Stock Quantity:") or product["stock"])
            save_data(PRODUCTS_FILE, products)
            messagebox.showinfo("Success", "Product updated successfully!")
            return
    messagebox.showerror("Error", "Product not found!")

# Admin will delete a product
def delete_product():
    product_id = simpledialog.askstring("Delete Product", "Enter Product ID to delete:")
    global products
    products = [product for product in products if product["id"] != product_id]
    save_data(PRODUCTS_FILE, products)
    messagebox.showinfo("Success", "Product deleted successfully!")

# Admin will view the sales report
def view_sales_report():
    time_period = simpledialog.askstring("View Sales Report", "Enter time period (daily, weekly, monthly):").lower()
    filtered_orders = []
    current_date = datetime.now()

    for order in orders:
        order_date = datetime.strptime(order["date"], "%Y-%m-%d %H:%M:%S")
        if time_period == "daily" and order_date.date() == current_date.date():
            filtered_orders.append(order)
        elif time_period == "weekly" and (current_date - order_date).days <= 7:
            filtered_orders.append(order)
        elif time_period == "monthly" and (current_date - order_date).days <= 30:
            filtered_orders.append(order)

    report = "\n".join([json.dumps(order, indent=4) for order in filtered_orders])
    messagebox.showinfo("Sales Report", report or "No orders found for the selected period.")

# Customer Functionalities
# Customers will browse the catalog
def browse_catalog():
    catalog = "\n".join([f"ID: {product['id']}, Name: {product['name']}, Price: {product['price']}, Stock: {product['stock']}" for product in products])
    messagebox.showinfo("Product Catalogue", catalog or "No products available.")

# Customers will place an order
def place_order():
    cart = []
    total_price = 0

    while True:
        product_id = simpledialog.askstring("Place Order", "Enter Product ID to add to cart (or type 'done' to finish):")
        if product_id.lower() == 'done':
            break

        quantity = int(simpledialog.askstring("Place Order", "Enter Quantity:"))
        for product in products:
            if product["id"] == product_id:
                if product["stock"] >= quantity:
                    product["stock"] -= quantity
                    cart.append({"id": product_id, "name": product["name"], "quantity": quantity, "price": product["price"]})
                    total_price += product["price"] * quantity
                else:
                    messagebox.showerror("Error", "Insufficient stock!")
                break

    order_summary = "\n".join([f"{item['name']} (x{item['quantity']}) - ${item['price'] * item['quantity']}" for item in cart])
    confirm = messagebox.askyesno("Order Summary", f"{order_summary}\nTotal Price: ${total_price}\nConfirm order?")

    if confirm:
        order_id = f"ORD-{len(orders) + 1}"
        order = {
            "id": order_id,
            "date": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
            "items": cart,
            "total_price": total_price
        }
        orders.append(order)
        save_data(ORDERS_FILE, orders)
        save_data(PRODUCTS_FILE, products)
        messagebox.showinfo("Success", f"Order placed successfully! Order ID: {order_id}")

# Customers will view their order history
def view_order_history():
    history = "\n".join([json.dumps(order, indent=4) for order in orders])
    messagebox.showinfo("Order History", history or "No order history available.")

# User Interface
# Admin menu will be created
def admin_menu():
    def open_admin_menu():
        menu = tk.Toplevel()
        menu.title("Admin Menu")

        tk.Button(menu, text="Add Product", command=add_product).pack(fill=tk.X)
        tk.Button(menu, text="Update Product", command=update_product).pack(fill=tk.X)
        tk.Button(menu, text="Delete Product", command=delete_product).pack(fill=tk.X)
        tk.Button(menu, text="View Sales Report", command=view_sales_report).pack(fill=tk.X)
        tk.Button(menu, text="Logout", command=menu.destroy).pack(fill=tk.X)

    open_admin_menu()

# Customer menu will be created
def customer_menu():
    def open_customer_menu():
        menu = tk.Toplevel()
        menu.title("Customer Menu")

        tk.Button(menu, text="Browse Catalogue", command=browse_catalog).pack(fill=tk.X)
        tk.Button(menu, text="Place Order", command=place_order).pack(fill=tk.X)
        tk.Button(menu, text="View Order History", command=view_order_history).pack(fill=tk.X)
        tk.Button(menu, text="Logout", command=menu.destroy).pack(fill=tk.X)

    open_customer_menu()

# Main interface will be set up
def main():
    root = tk.Tk()
    root.title("Online Ordering System")

    tk.Label(root, text="Welcome to the Online Ordering System", font=("Helvetica", 16)).pack(pady=10)
    tk.Button(root, text="Admin Login", command=admin_menu).pack(fill=tk.X, pady=5)
    tk.Button(root, text="Customer Login", command=customer_menu).pack(fill=tk.X, pady=5)
    tk.Button(root, text="Exit", command=root.destroy).pack(fill=tk.X, pady=5)

    root.mainloop()

if __name__ == "__main__":
    main()
 